<a href="https://colab.research.google.com/github/ghoshal7/PDF_extraction/blob/main/PDF_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/PDF_extraction

/content/drive/My Drive/PDF_extraction


In [3]:
%ls

 AGS-Lender-Presentation-201801.pdf
 Altisource_Lender_Presentation_March_2018.pdf
 Banijay-LenderPresentationPublic-TLBAmendandExtend.pdf
 Casino-Investor-presentation.pdf
 century_casinos.pdf
 entegris_8k.pdf
 Fluidra.pdf
 Holley-Credit-Investor-Presentation-October-2021.pdf
'IBP_Lender Presentation.pdf'
 ineos-quattro.pdf
 Kodak.pdf
'Topgolf Callaway Brands Corp Lender Presentation.pdf'
 Untitled0.ipynb


In [6]:
# !pip install PyPDF2
# !pip install pdfminer
# !pip install pdfplumber

import PyPDF2
from pdfminer.high_level import extract_text
import pdfplumber

In [98]:
def contains_table(text):
    # This is a basic heuristic.
    # You can refine this function based on more sophisticated patterns or rules.
    rows = text.split('\n')
    comma_counts = [row.count(',') for row in rows]
    # If multiple rows have multiple commas, it might indicate a table structure.
    potential_table_rows = sum(1 for count in comma_counts if count > 1)
    return potential_table_rows > 2

def find_pages_with_tables(pdf_path):
    pages_with_tables = []

    # Extract text from each page and check for tables
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        total_pages = len(reader.pages)

        for page_num in range(total_pages):
            text = extract_text(pdf_path, page_numbers=[page_num])
            if contains_table(text):
                # Page numbers usually start from 1 for users, so we add 1.
                pages_with_tables.append(page_num + 1)

    return pages_with_tables


def find_pages_with_pdfPlumber(pdf_path):
  pages_withSU = []

  import re
  pattern = re.compile(r"(?i)sources\s+(&|and)\s+uses")  # (?i) makes it case-insensitive

  with pdfplumber.open(pdf_path) as pdf:
    for page_num in range(len(pdf.pages)):
      page = pdf.pages[page_num]
      text = page.extract_text()
      if pattern.findall(text):
        # this page number starts with 0
        pages_withSU.append(page_num)

  return pages_withSU

In [102]:
pdf_path = "AGS-Lender-Presentation-201801.pdf"
pages_heuristic = find_pages_with_tables(pdf_path)
print(f"Pages with potential tables: {pages}")

pages_PP = find_pages_with_pdfPlumber(pdf_path)
print(f"Pages with REGEX SU: {pages_PP}")

Pages with potential tables: [2, 3, 7]
Pages with REGEX SU: [3]


In [107]:
def extract_term_loan_amount(pdf_path, pages_PP, keyword):
    with pdfplumber.open(pdf_path) as pdf:
      page = pdf.pages[pages_PP]
      text = page.filter(lambda obj:  (obj["object_type"] == "char" and obj["size"] > 7))
      texts = text.extract_text()
      for line in texts.split('\n'):
          if keyword in line:
              # Assuming the amount follows the keyword, split by spaces and get the next element
              # print(line.split())
              max_id = max([line.split().index(i) for i in keyword.split(' ')])
              amount = line.split()[max_id+1]
              return amount
      return None

In [109]:
pdf_path = "AGS-Lender-Presentation-201801.pdf"
pages_PP = find_pages_with_pdfPlumber(pdf_path)
print(pages_PP)
for key_word in ["Repriced 1st Lien Term Loan", "EBITDA", "Equipment Debt"]:
  print(key_word, " = ", extract_term_loan_amount(pdf_path, pages_PP[0], key_word))

[3]
Repriced 1st Lien Term Loan  =  $513
EBITDA  =  $104
Equipment Debt  =  3


In [110]:
pdf_path = "Altisource_Lender_Presentation_March_2018.pdf"
pages_PP = find_pages_with_pdfPlumber(pdf_path)

for key_word in ["Repriced 1st Lien Term Loan", "EBITDA", "Equipment Debt"]:
  print(key_word, " = ", extract_term_loan_amount(pdf_path, pages_PP[0], key_word))

Repriced 1st Lien Term Loan  =  None
EBITDA  =  Adj.
Equipment Debt  =  None


In [114]:
pdf_path = "Banijay-LenderPresentationPublic-TLBAmendandExtend.pdf"
pages_PP = find_pages_with_pdfPlumber(pdf_path)
print(pages_PP)
for key_word in ["Term Loan Facilities", "Cash from Balance Sheet", "Term Loan Facilities", "Fees and Expenses"]:
  print(key_word, " = ", extract_term_loan_amount(pdf_path, pages_PP[0], key_word))

[6]
Term Loan Facilities  =  895
Cash from Balance Sheet  =  9
Term Loan Facilities  =  895
Fees and Expenses  =  24
